# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east ballina,-28.8667,153.5833,12.11,61,20,3.85,AU,1720914461
1,1,devils lake,48.1128,-98.8651,28.30,62,0,5.14,US,1720914463
2,2,albany,42.6001,-73.9662,27.32,69,57,0.45,US,1720914464
3,3,new norfolk,-42.7826,147.0587,8.29,87,100,1.79,AU,1720914465
4,4,bilibino,68.0546,166.4372,16.08,52,100,2.13,RU,1720914466


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    size="Humidity", 
    alpha=0.5,
    frame_height=500, 
    scale= 1,
    color="City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [45]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df.loc[ (city_data_df['Max Temp'] < 28) & (city_data_df['Max Temp'] > 19) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
city_ideal_df.dropna()

# Display sample data
city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,jamestown,42.0970,-79.2353,26.80,50,0,2.57,US,1720914457
60,60,citta di castello,43.4691,12.2305,22.23,85,0,0.49,IT,1720914538
105,105,eruh,37.7517,42.1811,21.46,22,0,1.55,TR,1720914596
116,116,prince george,53.9166,-122.7530,26.46,36,0,2.06,CA,1720914428
199,199,darwin,-12.4611,130.8418,24.97,70,0,1.34,AU,1720914179
209,209,remire-montjoly,4.9167,-52.2667,25.02,94,0,3.09,GF,1720914725
218,218,orbetello,42.4408,11.2196,24.01,68,0,3.60,IT,1720914736
269,269,palmas de monte alto,-14.2672,-43.1619,21.46,50,0,4.34,BR,1720914799
270,270,neon karlovasion,37.7903,26.7044,25.14,78,0,3.49,GR,1720914800
278,278,bahar,34.9072,48.4414,20.09,40,0,1.03,IR,1720914810


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
41,jamestown,US,42.0970,-79.2353,50,
60,citta di castello,IT,43.4691,12.2305,85,
105,eruh,TR,37.7517,42.1811,22,
116,prince george,CA,53.9166,-122.7530,36,
199,darwin,AU,-12.4611,130.8418,70,
209,remire-montjoly,GF,4.9167,-52.2667,94,
218,orbetello,IT,42.4408,11.2196,68,
269,palmas de monte alto,BR,-14.2672,-43.1619,50,
270,neon karlovasion,GR,37.7903,26.7044,78,
278,bahar,IR,34.9072,48.4414,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey':geoapify_key,
    'format':'json',
    'categories':'accommodation.hotel',
    'filter':'circle',
    'bias':'proximity'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng_lat = f"{row['Lng']},{row['Lat']}"

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f"circle:{lng_lat},{radius}"
    params['bias'] = f"proximity:{lng_lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
citta di castello - nearest hotel: Hotel Park Geal
eruh - nearest hotel: No hotel found
prince george - nearest hotel: Travelodge by Wyndham Prince George
darwin - nearest hotel: Mantra Pantanas Darwin
remire-montjoly - nearest hotel: Complexe Belova
orbetello - nearest hotel: B&B La Pineta
palmas de monte alto - nearest hotel: No hotel found
neon karlovasion - nearest hotel: Ξενοδοχείο Αιγαίον
bahar - nearest hotel: هتل بوعلی
yorkton - nearest hotel: Holiday Inn Express
westport - nearest hotel: Norwalk Inn and Conference Center
puerto real - nearest hotel: Hostal Bahía de Cádiz
qarah - nearest hotel: No hotel found
milford - nearest hotel: Howard Johnson
qa'en - nearest hotel: No hotel found
el haouaria - nearest hotel: Villa Zembra Maison d'hôtes
dryden - nearest hotel: Holiday Inn Express
chernihiv - nearest hotel: Атланта
azov - nearest hotel: «АМАКС Отель Азов»
luchegorsk - nearest hotel: Мираж


,City,Country,Lat,Lng,Humidity,Hotel Name
41,jamestown,US,42.0970,-79.2353,50,DoubleTree Jamestown
60,citta di castello,IT,43.4691,12.2305,85,Hotel Park Geal
105,eruh,TR,37.7517,42.1811,22,No hotel found
116,prince george,CA,53.9166,-122.7530,36,Travelodge by Wyndham Prince George
199,darwin,AU,-12.4611,130.8418,70,Mantra Pantanas Darwin
209,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
218,orbetello,IT,42.4408,11.2196,68,B&B La Pineta
269,palmas de monte alto,BR,-14.2672,-43.1619,50,No hotel found
270,neon karlovasion,GR,37.7903,26.7044,78,Ξενοδοχείο Αιγαίον
278,bahar,IR,34.9072,48.4414,40,هتل بوعلی


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size='Humidity',
    scale = 1.3,
    color = 'City',
    hover_cols=['Humidity','Hotel Name', 'Country']
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)